# 하나의 구간에 대해

```

| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |  열차 하나 
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
                            .
                            .
                            .
                            .
                            .
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |  열차 백 

```
을 다 구해야 하는데 

```| ---- |```  이 부분 하나만 구하는 부분입니다.

In [26]:
import random
import math
import re
import operator 
from random import randint

# 리스트 크기
listN = 8

# 염색체 해집단 수
N = 30

#교차율
Pc = 0.7
#변이율 역수 관계
Pm = 1000

#마지막 세대수 
largest_generation = 500

#보존할 상위값
save_value = 5



In [27]:

def decoding(X, Y=0):
    #2번째 값부터 2진수를 10진수 화 (0부터 시작 -1, 처음값 제외 -1)
    for i, x in enumerate(X[1:]):
        Y += x* math.pow(2, listN -i -2)
    #처음 값이 0이면 음수로 return
    if X[0] == 0: return -Y
    return Y

In [28]:
def Readtxt(fname):
    #파일을 2차원 배열로 바꿔주기
    w = []
    with open(fname, 'r') as f :
        for line in f: #for each line in the file,
            w.append(line.split('\t'))
    return w

In [29]:
def ScoreFunction(X):
    """
    X에 속도 리스트를 넣어서 평가
    """
    AirportNum = Readtxt('data/Number-of-train-people.txt')
    AirportTime = Readtxt('data/Time-of-airport-train.txt')
    AtherTime = Readtxt('data/Time-of-ather-train.txt')
    sp = decoding(X)
    return 777 - (6 * sp)

In [30]:
def New_ScoreFunction(X):
    #여기다 바꾼 거를 만들면 됩니다.
    print(X)

In [31]:
#초기값
speedlist = [[random.choice([0,1]) for i in range(listN)]for j in range(N)]
speedlist

for sl in speedlist:
    print(ScoreFunction(sl), sl)

801.0 [0, 0, 0, 0, 0, 1, 0, 0]
669.0 [1, 0, 0, 1, 0, 0, 1, 0]
381.0 [1, 1, 0, 0, 0, 0, 1, 0]
513.0 [1, 0, 1, 0, 1, 1, 0, 0]
405.0 [1, 0, 1, 1, 1, 1, 1, 0]
1155.0 [0, 0, 1, 1, 1, 1, 1, 1]
933.0 [0, 0, 0, 1, 1, 0, 1, 0]
1191.0 [0, 1, 0, 0, 0, 1, 0, 1]
243.0 [1, 1, 0, 1, 1, 0, 0, 1]
1095.0 [0, 0, 1, 1, 0, 1, 0, 1]
477.0 [1, 0, 1, 1, 0, 0, 1, 0]
921.0 [0, 0, 0, 1, 1, 0, 0, 0]
309.0 [1, 1, 0, 0, 1, 1, 1, 0]
1101.0 [0, 0, 1, 1, 0, 1, 1, 0]
1317.0 [0, 1, 0, 1, 1, 0, 1, 0]
933.0 [0, 0, 0, 1, 1, 0, 1, 0]
147.0 [1, 1, 1, 0, 1, 0, 0, 1]
495.0 [1, 0, 1, 0, 1, 1, 1, 1]
273.0 [1, 1, 0, 1, 0, 1, 0, 0]
1521.0 [0, 1, 1, 1, 1, 1, 0, 0]
297.0 [1, 1, 0, 1, 0, 0, 0, 0]
873.0 [0, 0, 0, 1, 0, 0, 0, 0]
831.0 [0, 0, 0, 0, 1, 0, 0, 1]
771.0 [1, 0, 0, 0, 0, 0, 0, 1]
705.0 [1, 0, 0, 0, 1, 1, 0, 0]
519.0 [1, 0, 1, 0, 1, 0, 1, 1]
1317.0 [0, 1, 0, 1, 1, 0, 1, 0]
423.0 [1, 0, 1, 1, 1, 0, 1, 1]
1503.0 [0, 1, 1, 1, 1, 0, 0, 1]
1329.0 [0, 1, 0, 1, 1, 1, 0, 0]


In [32]:
# 여기서 부터 세대 돌리게 만들어야 됨

def Group_assessment(speedlist):
    #speedlist, decoding값, 염색체 적합도, 적합도 비율
    speedlist_evaluation = [[speedlist[i], decoding(speedlist[i]), ScoreFunction(speedlist[i]), 0,i] for i in range(N)]
    # 적합도 비율 찾기
    # (각자의 값+이전값 총합)/총합
    Sum_of_fitness = 0
    for i in range(N): 
        Sum_of_fitness += speedlist_evaluation[i][2]
    for i in range(N):
        if i == 0:
            speedlist_evaluation[i][3] = speedlist_evaluation[i][2]/Sum_of_fitness
        else:
            speedlist_evaluation[i][3] = speedlist_evaluation[i-1][3]  + speedlist_evaluation[i][2]/Sum_of_fitness
    
    return speedlist_evaluation


Group_assessment_value = Group_assessment(speedlist)
#Group_assessment_value
sortand = sorted(Group_assessment_value, key=operator.itemgetter(2),reverse=True)
save_assessmend_value = sortand[0:save_value]
save_assessmend_num = [save_assessmend_value[i][4] for i in range(save_value)]
print(save_assessmend_num)

save_assessmend_value

[19, 28, 29, 14, 26]


[[[0, 1, 1, 1, 1, 1, 0, 0], -124.0, 1521.0, 0.6345957011258957, 19],
 [[0, 1, 1, 1, 1, 0, 0, 1], -121.0, 1503.0, 0.9433213920163768, 28],
 [[0, 1, 0, 1, 1, 1, 0, 0], -92.0, 1329.0, 1.0, 29],
 [[0, 1, 0, 1, 1, 0, 1, 0], -90.0, 1317.0, 0.4909160696008189, 14],
 [[0, 1, 0, 1, 1, 0, 1, 0], -90.0, 1317.0, 0.8611821903787105, 26]]

In [33]:
# 0 ~ 100 룰랫 돌려서 이전 값과 자신 사이에 있으면 선택
def roulettes(Group_assessment_value):
    roulette = random.random()
    for i in range(N):
        if i == 0:
            if 0 < roulette < Group_assessment_value[i][3]:
                break
        if Group_assessment_value[i-1][3] < roulette < Group_assessment_value[i][3]:
            break
    return i

In [40]:
#부모 변경
def Change_parent(speedlist, Selected_list_in_roulette):
    W = []
    # 교차함수 및 변이함수 (자식만들기)
    for i in range(int(N/2)):
        #교차
        selection = random.random()
        if selection < Pc:
            rand = random.choice([i+1 for i in range(listN-1)]) # 0~ 이구간, 이구간 ~ 끝 교차 , 이구간 = 1~리스트 크기 -1 
            # W(교차된 자식) = Selected_list_in_roulette(선택된 리스트에서) rand구간으로 나눈 값
            W.append(speedlist[Selected_list_in_roulette[i*2]][0:rand] + speedlist[Selected_list_in_roulette[i*2+1]][rand:])
            W.append(speedlist[Selected_list_in_roulette[i*2+1]][0:rand] + speedlist[Selected_list_in_roulette[i*2]][rand:])
            #print(i, W)
        else:
            W.append(speedlist[Selected_list_in_roulette[i*2]])
            W.append(speedlist[Selected_list_in_roulette[i*2+1]])
            #print(i, W)
        
        #변의
        for j in range(2):
            # Pm * 100 분의 1의 확률 
            if randint(0, 100 * Pm) == 0:
                #selection 부분을 변의 1 -> 0  0 -> 1
                W[i*2+j][selection] = -1 * (W[i*2+j][selection] -1)
                    
    return W

In [41]:
#세대 반복
def Evolution(speedlist, num_generation):
    while 1: 
        if num_generation >= largest_generation:
            break

        print("-" * 15)    
        print(speedlist)
        print("-" * 15)    

        #적합도 찾고 룰렛 준비
        Group_assessment_value = Group_assessment(speedlist)
        #상위 5개값
        sortand_Group_assessmend = sorted(Group_assessment_value, key=operator.itemgetter(2),reverse=True)
        save_assessmend_value = sortand_Group_assessmend[0:save_value]
        save_assessmend_num = [save_assessmend_value[i][4] for i in range(save_value)]
        #룰렛에 선택된 값 찾기 
        Selected_list_in_roulette = []

        #룰렛에 선택된 리스트들 마지막 5개는 상위5개만 
        for i in range(N-5):
            Selected_list_in_roulette.append(roulettes(Group_assessment_value))
        
        Selected_list_in_roulette += save_assessmend_num
        
        New_list = Change_parent(speedlist, Selected_list_in_roulette)
        print("-" * 15)    
        print(num_generation,"번째를 합니다.")
        print("-" * 15)
        for i, nl in enumerate(New_list):
            print(i, ScoreFunction(nl), nl)
        
        speedlist = New_list
        num_generation += 1

    return New_list

   

In [42]:
test_list = []

test_list = Evolution(speedlist, 0)

---------------
[[0, 0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 1, 0, 0, 1, 0], [1, 1, 0, 0, 0, 0, 1, 0], [1, 0, 1, 0, 1, 1, 0, 0], [1, 0, 1, 1, 1, 1, 1, 0], [0, 0, 1, 1, 1, 1, 1, 1], [0, 0, 0, 1, 1, 0, 1, 0], [0, 1, 0, 0, 0, 1, 0, 1], [1, 1, 0, 1, 1, 0, 0, 1], [0, 0, 1, 1, 0, 1, 0, 1], [1, 0, 1, 1, 0, 0, 1, 0], [0, 0, 0, 1, 1, 0, 0, 0], [1, 1, 0, 0, 1, 1, 1, 0], [0, 0, 1, 1, 0, 1, 1, 0], [0, 1, 0, 1, 1, 0, 1, 0], [0, 0, 0, 1, 1, 0, 1, 0], [1, 1, 1, 0, 1, 0, 0, 1], [1, 0, 1, 0, 1, 1, 1, 1], [1, 1, 0, 1, 0, 1, 0, 0], [0, 1, 1, 1, 1, 1, 0, 0], [1, 1, 0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 1], [1, 0, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 1, 1, 0, 0], [1, 0, 1, 0, 1, 0, 1, 1], [0, 1, 0, 1, 1, 0, 1, 0], [1, 0, 1, 1, 1, 0, 1, 1], [0, 1, 1, 1, 1, 0, 0, 1], [0, 1, 0, 1, 1, 1, 0, 0]]
---------------
---------------
0 번째를 합니다.
---------------
0 669.0 [1, 0, 0, 1, 0, 0, 1, 0]
1 1329.0 [0, 1, 0, 1, 1, 1, 0, 0]
2 1155.0 [0, 0, 1, 1, 1, 1, 1, 1]
3 297.0 [1, 1, 0, 1, 0, 0, 0, 0]
4 1155.0 [

22 1509.0 [0, 1, 1, 1, 1, 0, 1, 0]
23 1521.0 [0, 1, 1, 1, 1, 1, 0, 0]
24 1533.0 [0, 1, 1, 1, 1, 1, 1, 0]
25 1521.0 [0, 1, 1, 1, 1, 1, 0, 0]
26 1533.0 [0, 1, 1, 1, 1, 1, 1, 0]
27 1533.0 [0, 1, 1, 1, 1, 1, 1, 0]
28 1527.0 [0, 1, 1, 1, 1, 1, 0, 1]
29 1527.0 [0, 1, 1, 1, 1, 1, 0, 1]
---------------
[[0, 1, 1, 1, 1, 1, 0, 0], [0, 1, 1, 1, 1, 1, 0, 0], [0, 1, 1, 1, 1, 1, 0, 0], [0, 1, 1, 1, 1, 1, 0, 0], [0, 1, 1, 1, 1, 1, 0, 0], [0, 1, 1, 1, 1, 0, 1, 0], [0, 1, 1, 1, 1, 1, 0, 0], [0, 1, 1, 1, 1, 1, 0, 1], [0, 1, 1, 1, 0, 0, 0, 0], [0, 1, 1, 1, 1, 1, 0, 0], [0, 1, 1, 1, 1, 0, 1, 0], [0, 1, 1, 1, 1, 1, 0, 0], [0, 1, 1, 1, 1, 0, 0, 0], [0, 1, 1, 1, 0, 1, 0, 1], [0, 1, 1, 1, 1, 1, 0, 1], [0, 1, 0, 1, 1, 1, 0, 0], [0, 1, 0, 1, 1, 1, 0, 0], [0, 1, 1, 1, 1, 0, 1, 0], [0, 1, 1, 1, 1, 1, 0, 0], [0, 1, 1, 1, 1, 1, 0, 0], [0, 1, 0, 1, 1, 1, 0, 1], [0, 1, 1, 1, 1, 0, 1, 1], [0, 1, 1, 1, 1, 0, 1, 0], [0, 1, 1, 1, 1, 1, 0, 0], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 0, 0], [0, 1, 1, 1, 1, 1, 1, 0], [

28 1533.0 [0, 1, 1, 1, 1, 1, 1, 0]
29 1533.0 [0, 1, 1, 1, 1, 1, 1, 0]
---------------
[[0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 0, 1, 0], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 0, 1], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 0, 1, 0], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 0]]
---------------
---------------
12 번째를 합니다.
---------------
0 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
1 1521.0 [0, 1, 1, 1, 1, 1, 0, 0]
2 153

18 1533.0 [0, 1, 1, 1, 1, 1, 1, 0]
19 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
20 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
21 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
22 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
23 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
24 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
25 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
26 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
27 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
28 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
29 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
---------------
[[0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1

---------------
24 번째를 합니다.
---------------
0 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
1 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
2 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
3 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
4 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
5 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
6 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
7 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
8 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
9 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
10 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
11 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
12 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
13 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
14 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
15 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
16 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
17 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
18 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
19 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
20 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
21 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
22 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
23 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
24 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
25 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
26 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
27 1539.0 [0, 1, 1, 1

...

1 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
2 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
3 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
4 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
5 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
6 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
7 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
8 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
9 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
10 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
11 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
12 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
13 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
14 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
15 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
16 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
17 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
18 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
19 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
20 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
21 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
22 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
23 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
24 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
25 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
26 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
27 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
28 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
29 1539.0 [0, 1, 1, 1, 1, 1, 

12 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
13 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
14 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
15 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
16 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
17 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
18 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
19 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
20 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
21 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
22 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
23 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
24 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
25 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
26 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
27 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
28 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
29 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
---------------
[[0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1,

1 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
2 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
3 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
4 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
5 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
6 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
7 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
8 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
9 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
10 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
11 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
12 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
13 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
14 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
15 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
16 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
17 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
18 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
19 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
20 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
21 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
22 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
23 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
24 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
25 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
26 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
27 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
28 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
29 1539.0 [0, 1, 1, 1, 1, 1, 

5 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
6 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
7 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
8 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
9 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
10 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
11 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
12 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
13 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
14 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
15 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
16 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
17 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
18 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
19 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
20 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
21 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
22 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
23 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
24 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
25 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
26 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
27 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
28 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
29 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
---------------
[[0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1,

---------------
495 번째를 합니다.
---------------
0 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
1 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
2 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
3 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
4 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
5 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
6 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
7 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
8 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
9 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
10 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
11 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
12 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
13 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
14 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
15 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
16 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
17 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
18 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
19 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
20 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
21 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
22 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
23 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
24 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
25 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
26 1539.0 [0, 1, 1, 1, 1, 1, 1, 1]
27 1539.0 [0, 1, 1, 

In [39]:
print(test_list)

[[0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1]]


In [2]:
a = Readtxt( 'data/Number-of-train-people.txt')

In [3]:
a[0][3]

'17683'

In [4]:
a

[['서울',
  '일반',
  '9516',
  '17683',
  '27199',
  '8170',
  '18430',
  '26600',
  '53799',
  '\n'],
 ['공덕',
  '일반',
  '1674',
  '12842',
  '14516',
  '2140',
  '13366',
  '15506',
  '30022',
  '\n'],
 ['홍대',
  '일반',
  '9088',
  '19332',
  '28420',
  '9007',
  '21731',
  '30738',
  '59158',
  '\n'],
 ['미디어시티',
  '일반',
  '11639',
  '11423',
  '23062',
  '10200',
  '12101',
  '22301',
  '45363',
  '\n'],
 ['김포공항',
  '일반',
  '10393',
  '30080',
  '40473',
  '6548',
  '34747',
  '41295',
  '81768',
  '\n'],
 ['계양',
  '일반',
  '10865',
  '18624',
  '29489',
  '10130',
  '18776',
  '28906',
  '58395',
  '\n'],
 ['검암',
  '일반',
  '9144',
  '14121',
  '23265',
  '9039',
  '12846',
  '21885',
  '45150',
  '\n'],
 ['청라', '일반', '5323', '0', '5323', '5121', '0', '5121', '10444', '\n'],
 ['영종', '일반', '1869', '0', '1869', '1831', '0', '1831', '3700', '\n'],
 ['운서', '일반', '5724', '0', '5724', '5795', '0', '5795', '11519', '\n'],
 ['화물청사', '일반', '2619', '0', '2619', '2899', '0', '2899', '5518', '\n'],
 [